In [19]:
#importing the libraries
import torch
import numpy as np
import matplotlib.pyplot as plt

In [20]:
from sklearn.datasets import load_diabetes
import numpy as np

#loading the dataset
data = load_diabetes()

print(data.DESCR)

# get everything to describe the dataset
print(data.keys())


.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

:Number of Instances: 442

:Number of Attributes: First 10 columns are numeric predictive values

:Target: Column 11 is a quantitative measure of disease progression one year after baseline

:Attribute Information:
    - age     age in years
    - sex
    - bmi     body mass index
    - bp      average blood pressure
    - s1      tc, total serum cholesterol
    - s2      ldl, low-density lipoproteins
    - s3      hdl, high-density lipoproteins
    - s4      tch, total cholesterol / HDL
    - s5      ltg, possibly log of serum triglycerides level
    - s6      glu, blood sugar level

Note: Each of these 10 feature variables have bee

In [21]:
import pandas as pd

x = data['data']

y = data['target']

df = pd.DataFrame(x, columns=data['feature_names'])
df['target'] = y

# print 

# shape of the data
print(x.shape)
print(y.shape)

(442, 10)
(442,)


In [47]:
from torch.utils.data import DataLoader, Dataset
import pandas as pd

class DiabetesDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.length = self.x.shape[0]

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
    
    def __len__(self):
        return self.length
    
dataset = DiabetesDataset(x, y)

# dataset.__getitem__(torch.randint(0, dataset.length, (1,)).item())
# pd.DataFrame(dataset.x.numpy())

,0,1,2,3,4,5,6,7,8,9
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641
...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018114,0.044485
439,0.041708,0.050680,-0.015906,0.017293,-0.037344,-0.013840,-0.024993,-0.011080,-0.046883,0.015491
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044529,-0.025930


In [222]:
# used for minibatch/stochastic gradient descent by acting as an iterable object
data_loader = DataLoader(dataset, batch_size=100, shuffle=True)

In [225]:
# lets create the network

from torch import nn

class net(nn.Module):
    def __init__(self, input_size, output_size):
        super(net, self).__init__()
        self.l1 = nn.Linear(input_size, 5)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(5, output_size)

    def forward(self, x):
        output = self.l1(x)
        output = self.relu(output)
        output = self.l2(output)
        return output 
    
model = net(x.shape[1], 1)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
epochs = 1500



In [228]:
costval = []

for j in range(epochs):
    for i, (x_train, y_train) in enumerate(data_loader):
        # prediction
        y_pred = model(x_train)

        # calculating the loss 
        cost = criterion(y_pred, y_train.reshape(-1,1))

        # backprop
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if j%10 ==0:
            print(cost)
            costval.append(cost)

print(costval)

tensor(3764.1641, grad_fn=<MseLossBackward0>)
tensor(3032.8125, grad_fn=<MseLossBackward0>)
tensor(2438.6746, grad_fn=<MseLossBackward0>)
tensor(2464.4697, grad_fn=<MseLossBackward0>)
tensor(2647.0708, grad_fn=<MseLossBackward0>)
tensor(3083.1282, grad_fn=<MseLossBackward0>)
tensor(3362.3899, grad_fn=<MseLossBackward0>)
tensor(2837.8582, grad_fn=<MseLossBackward0>)
tensor(2695.5632, grad_fn=<MseLossBackward0>)
tensor(2260.3628, grad_fn=<MseLossBackward0>)
tensor(3010.8928, grad_fn=<MseLossBackward0>)
tensor(2655.6416, grad_fn=<MseLossBackward0>)
tensor(2800.3193, grad_fn=<MseLossBackward0>)
tensor(3179.9629, grad_fn=<MseLossBackward0>)
tensor(2767.6038, grad_fn=<MseLossBackward0>)
tensor(3319.7478, grad_fn=<MseLossBackward0>)
tensor(2539.9434, grad_fn=<MseLossBackward0>)
tensor(2890.7432, grad_fn=<MseLossBackward0>)
tensor(2793.7971, grad_fn=<MseLossBackward0>)
tensor(2874.6641, grad_fn=<MseLossBackward0>)
tensor(2489.7727, grad_fn=<MseLossBackward0>)
tensor(3057.4116, grad_fn=<MseLoss

In [ ]:

from pathlib import Path
import requests

DATA_PATH = Path("data")
PATH = DATA_PATH / "mnist"

PATH.mkdir(parents=True, exist_ok=True)

URL = "https://github.com/pytorch/tutorials/raw/main/_static/"
FILENAME = "mnist.pkl.gz"

if not (PATH / FILENAME).exists():
        content = requests.get(URL + FILENAME).content
        (PATH / FILENAME).open("wb").write(content)